In [20]:
import theano
import theano.tensor as T
import numpy as np

a = T.tensor3()
b = T.matrix()

b_re = b.reshape([b.shape[0], 1, b.shape[1]])

br_zeros = T.zeros_like(a)

b_br = b_re + br_zeros
test1 = theano.function([a, b], b_br)
result = T.batched_dot(a, b_re.dimshuffle(0,2,1))
mask = T.matrix()
final_result = result * mask.reshape([mask.shape[0], mask.shape[1], 1])
test = theano.function([a, b, mask], final_result)

In [21]:
a_1 = np.array([[[1,2,3], [4,5,6], [7,8,9], [10,11,12]],[[13,14,15],[16,17,18],[19,20,21],[22,23,24]]])
b_1 = np.array([[1,1,1], [2,2,2]])
mask_1 = np.array([[1,1,1,1],[1,1,1,0]])
test(a_1, b_1, mask_1)

array([[[   6.],
        [  15.],
        [  24.],
        [  33.]],

       [[  84.],
        [ 102.],
        [ 120.],
        [   0.]]])

In [14]:
a = T.matrix()
b = T.tensor3()

c = a*b

result = theano.function([a,b],c)

In [15]:
a_ = np.array([[1,2,3],[4,4,4]])
b_ = np.array([[[1],[1],[1]],[[2],[2],[2]]])
result(a_, b_)

ValueError: Input dimension mis-match. (input[0].shape[1] = 2, input[1].shape[1] = 3)
Apply node that caused the error: Elemwise{mul,no_inplace}(InplaceDimShuffle{x,0,1}.0, <TensorType(float64, 3D)>)
Toposort index: 1
Inputs types: [TensorType(float64, (True, False, False)), TensorType(float64, 3D)]
Inputs shapes: [(1, 2, 3), (2, 3, 1)]
Inputs strides: [(48, 24, 8), (24, 8, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [['output']]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    user_expressions, allow_stdin)
  File "/usr/local/lib/python2.7/site-package

In [6]:
import lasagne
import theano
import theano.tensor as T
a = T.vector()
b = T.vector()

c = lasagne.objectives.binary_crossentropy(a, b)
test = theano.function([a,b], c)



In [8]:
import numpy as np
a = np.array([0.9,0.5,0.2])
b = np.array([0,1,1])
test(a,b)

array([ 2.30258509,  0.69314718,  1.60943791])

In [12]:
a = T.tensor3()
b = T.matrix()

c = T.mean(a, axis = 1)
d = T.sum(a, axis = 1) / T.sum(b, axis = 1).reshape([-1,1])

test1 = theano.function([a,b], [c,d])



In [14]:
a_1 = np.array([[[1,2,3], [4,5,6], [7,8,9], [10,11,12]],[[13,14,15],[16,17,18],[19,20,21],[22,23,24]]])
b_1 = np.array([[1,1,1], [2,2,2]])
mask_1 = np.array([[1,1,1,1],[1,1,1,0]])
test1(a_1, mask_1)

[array([[  5.5,   6.5,   7.5],
        [ 17.5,  18.5,  19.5]]),
 array([[  5.5       ,   6.5       ,   7.5       ],
        [ 23.33333333,  24.66666667,  26.        ]])]

In [402]:
import BLSTM_sequence
import sys
# from theano.printing import pydotprint




class Hierachi_RNN(object):
    def __init__(self, rnn_setting, dropout_rate, batchsize, val_split_ratio, wemb_size = None):
        # Initialize Theano Symbolic variable attributes
        self.story_input_variable = None
        self.story_mask = None
        self.story_nsent = 4

        self.cost = None

        self.train_func = None

        # Initialize data loading attributes
        self.wemb = theano.shared(np.random.rand(200, 300))
        self.val_set_path = '../../data/pickles/val_index_corpus.pkl'
        self.test_set_path = '../../data/pickles/test_index_corpus.pkl' 

        self.wemb_matrix_path = '../../data/pickles/index_wemb_matrix.pkl'

        self.rnn_units = int(rnn_setting)
        # self.mlp_units = [int(elem) for elem in mlp_setting.split('x')]
        self.bilinear_matrix = theano.shared(0.002*np.random.rand(self.rnn_units, self.rnn_units)-0.001)
        self.dropout_rate = float(dropout_rate)
        self.batchsize = int(batchsize)

        self.val_split_ratio = float(val_split_ratio)

        self.wemb_size = 300
        if wemb_size == None:
            self.random_init_wemb = False
        else:
            self.random_init_wemb = True
            self.wemb_size = int(wemb_size)

        self.train_story = None
        self.train_ending = None

        self.val_story = None
        self.val_ending1 = None 
        self.val_ending2 = None
        self.val_answer = None
        self.n_val = None

        self.test_story = None 
        self.test_ending1 = None
        self.test_ending2 = None
        self.test_answer = None
        self.n_test = None

        self.train_encodinglayer_vecs = []
        self.test_encodinglayer_vecs = []
        self.reasoninglayer_vec1 = []
        self.reasoninglayer_vec2 = []
        self.reasoninglayer_vec1_test = []
        self.reasoninglayer_vec2_test = []
        self.reasoning_pool_results = []
        self.reasoning_pool_results_test = []
        self.reasoners = []
        self.attentioned_sent_rep1 = []
        self.attentioned_sent_rep2 = []
        self.monitor1 = []
        self.monitor2 = []
        self.attention_moni1 = []
        self.attention_moni2 = []
        self.softmask_moni = []
        self.bilinear_attention_matrix = theano.shared(0.02*np.random.rand(self.rnn_units, self.rnn_units) - 0.01)

        
    def encoding_layer(self):


        assert len(self.reshaped_inputs_variables)==len(self.inputs_masks)
        for i in range(self.story_nsent):
            self.train_encodinglayer_vecs.append(lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[i], 
                                                         self.encoder.l_mask:self.inputs_masks[i]},
                                                         deterministic = True))
        ending1_sequence_tensor = lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[4],
                                                        self.encoder.l_mask:self.inputs_masks[4]},
                                                        deterministic = True)
        ending2_sequence_tensor = lasagne.layers.get_output(self.encoder.output,
                                                        {self.encoder.l_in:self.reshaped_inputs_variables[5],
                                                        self.encoder.l_mask:self.inputs_masks[5]},
                                                        deterministic = True)

        l_end_in= lasagne.layers.InputLayer((None, None, self.rnn_units))
        l_shuffle = lasagne.layers.DimshuffleLayer(l_end_in, (0,2,1))
        l_pooling = lasagne.layers.GlobalPoolLayer(l_shuffle)


        end1_representation = lasagne.layers.get_output(l_pooling, {l_end_in:ending1_sequence_tensor})
        end2_representation = lasagne.layers.get_output(l_pooling, {l_end_in:ending2_sequence_tensor})
        self.train_encodinglayer_vecs.append(end1_representation)
        self.train_encodinglayer_vecs.append(end2_representation)
        
    def attention_layer(self):        
        for i in range(self.story_nsent):
            n_batch, n_seq, _ = self.train_encodinglayer_vecs[i].shape

      
            bili_part1 = T.dot(self.train_encodinglayer_vecs[i], self.bilinear_attention_matrix)

            attention1_score_tensor = T.batched_dot(bili_part1, self.train_encodinglayer_vecs[4])

            attention2_score_tensor = T.batched_dot(bili_part1, self.train_encodinglayer_vecs[5])

#             softmax_mask = 
#             self.softmask_moni.append(softmax_mask)
            numerator1 = self.inputs_masks[i] * T.exp(attention1_score_tensor - attention1_score_tensor.max(axis = 1, keepdims = True))
            numerator2 = self.inputs_masks[i] * T.exp(attention2_score_tensor - attention2_score_tensor.max(axis = 1, keepdims = True))
        
            attention1_weight_matrix = numerator1 / numerator1.sum(axis = 1, keepdims = True)
            attention2_weight_matrix = numerator2 / numerator2.sum(axis = 1, keepdims = True)
#             attention1_weight_matrix = T.nnet.softmax(attention1_score_tensor.reshape([n_batch, -1])*self.inputs_masks[i])
#             attention2_weight_matrix = T.nnet.softmax(attention2_score_tensor.reshape([n_batch, -1])*self.inputs_masks[i])
            self.attention_moni1.append(attention1_weight_matrix)
            self.attention_moni2.append(attention2_weight_matrix)
            
            attentioned_sent_seq1 = self.train_encodinglayer_vecs[i]*(attention1_weight_matrix.reshape([n_batch, n_seq, 1]))
            attentioned_sent_seq2 = self.train_encodinglayer_vecs[i]*(attention2_weight_matrix.reshape([n_batch, n_seq, 1]))
            
            self.monitor1.append(attentioned_sent_seq1)
            self.monitor2.append(attentioned_sent_seq2)
            
            attentioned_sent_rep1 = T.sum(attentioned_sent_seq1, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])
            attentioned_sent_rep2 = T.sum(attentioned_sent_seq2, axis = 1) / T.sum(self.inputs_masks[i], axis = 1).reshape([-1, 1])

            self.attentioned_sent_rep1.append(attentioned_sent_rep1)
            self.attentioned_sent_rep2.append(attentioned_sent_rep2)

           
    def model_constructor(self, wemb_size = None):
        self.inputs_variables = []
        self.inputs_masks = []
        self.reshaped_inputs_variables = []
        for i in range(self.story_nsent+2):
            self.inputs_variables.append(T.matrix('story'+str(i)+'_input', dtype='int64'))
            self.inputs_masks.append(T.matrix('story'+str(i)+'_mask', dtype=theano.config.floatX))
            batch_size, seqlen = self.inputs_variables[i].shape
            self.reshaped_inputs_variables.append(self.inputs_variables[i].reshape([batch_size, seqlen, 1]))

        #initialize neural network units
        self.encoder = BLSTM_sequence.BlstmEncoder(LSTMLAYER_1_UNITS = self.rnn_units, dropout_rate = self.dropout_rate)
        self.encoder.build_model(self.wemb)

        #build encoding layer
        self.encoding_layer()
        self.test0 = theano.function(self.inputs_variables + self.inputs_masks, self.train_encodinglayer_vecs)
        #build attention layer
        self.attention_layer()
        
        self.test2 = theano.function(self.inputs_variables + self.inputs_masks, self.monitor1+self.monitor2)
        self.test3 = theano.function(self.inputs_variables + self.inputs_masks, self.attention_moni1 + self.attention_moni2)
        
#         self.test4 = theano.function(self.inputs_masks[:4], self.softmask_moni)
        #build reasoning layers
        
        self.test1 = theano.function(self.inputs_variables + self.inputs_masks, self.attentioned_sent_rep1 + self.attentioned_sent_rep2)
        self.merge_ls1 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep1]
        self.merge_ls2 = [T.reshape(tensor, (tensor.shape[0], 1, tensor.shape[1])) for tensor in self.attentioned_sent_rep2]

        encode_merge1 = T.concatenate(self.merge_ls1, axis = 1)
        encode_merge2 = T.concatenate(self.merge_ls2, axis = 1)

        l_in = lasagne.layers.InputLayer(shape=(None, None, self.rnn_units))
        gate_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                        W_hid=lasagne.init.Orthogonal(),
                                                        b=lasagne.init.Constant(0.))

        cell_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                        W_hid=lasagne.init.Orthogonal(),
                                                        # Setting W_cell to None denotes that no cell connection will be used. 
                                                        W_cell=None, 
                                                        b=lasagne.init.Constant(0.),
                                                        # By convention, the cell nonlinearity is tanh in an LSTM. 
                                                        nonlinearity=lasagne.nonlinearities.tanh)

        l_lstm = lasagne.layers.recurrent.LSTMLayer(l_in, 
                                                    num_units=self.rnn_units,
                                                    # Here, we supply the gate parameters for each gate 
                                                    ingate=gate_parameters, forgetgate=gate_parameters, 
                                                    cell=cell_parameters, outgate=gate_parameters,
                                                    # We'll learn the initialization and use gradient clipping 
                                                    learn_init=True, grad_clipping=100.0)

        # The back directional LSTM layers
        l_lstm_back = lasagne.layers.recurrent.LSTMLayer(l_in,
                                                         num_units=self.rnn_units,
                                                         ingate=gate_parameters, forgetgate=gate_parameters, 
                                                         cell=cell_parameters, outgate=gate_parameters,
                                                         # We'll learn the initialization and use gradient clipping 
                                                         learn_init=True,grad_clipping=100.0,
                                                         backwards=True)

        # Do sum up of bidirectional LSTM results
        l_out_right = lasagne.layers.SliceLayer(l_lstm, -1, 1)
        l_out_left = lasagne.layers.SliceLayer(l_lstm_back, -1, 1)
        l_sum = lasagne.layers.ElemwiseSumLayer([l_out_right, l_out_left])

        reasoner_result1 = lasagne.layers.get_output(l_sum, {l_in: encode_merge1}, deterministic = True)
        reasoner_result2 = lasagne.layers.get_output(l_sum, {l_in: encode_merge2}, deterministic = True)

        reasoner_params = lasagne.layers.get_all_params(l_sum)

        # l_story_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        # l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        # l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)

        # l_hid = lasagne.layers.DenseLayer(l_concate, num_units=2,
        #                                   nonlinearity=lasagne.nonlinearities.tanh)

        # final_class_param = lasagne.layers.get_all_params(l_hid)
        
        l_story_in = lasagne.layers.InputLayer(shape=(None, self.rnn_units))
        l_end_in = lasagne.layers.InputLayer(shape = (None, self.rnn_units))
        l_concate = lasagne.layers.ConcatLayer([l_story_in, l_end_in], axis = 1)

        l_hid = lasagne.layers.DenseLayer(l_concate, num_units=2,
                                          nonlinearity=lasagne.nonlinearities.tanh)

        final_class_param = lasagne.layers.get_all_params(l_hid)

        score1 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result1, 
                                                   l_end_in: self.train_encodinglayer_vecs[-2]})
        score2 = lasagne.layers.get_output(l_hid, {l_story_in: reasoner_result2, 
                                                   l_end_in: self.train_encodinglayer_vecs[-1]})
        self.test5 = theano.function(self.inputs_variables + self.inputs_masks,[score1, score2])
        prob1 = lasagne.nonlinearities.softmax(score1)
        prob2 = lasagne.nonlinearities.softmax(score2)

        # Construct symbolic cost function
        target1 = T.vector('gold_target1', dtype= 'int64')
        target2 = T.vector('gold_target2', dtype= 'int64')
        
        cost1 = lasagne.objectives.categorical_crossentropy(prob1, target1)
        cost2 = lasagne.objectives.categorical_crossentropy(prob2, target2)

        self.cost = lasagne.objectives.aggregate(cost1+cost2, mode='sum')

        # Retrieve all parameters from the network
        all_params = self.encoder.all_params + reasoner_params + final_class_param

        all_updates = lasagne.updates.sgd(self.cost, all_params, learning_rate=0.001)
        # all_updates = lasagne.updates.momentum(self.cost, all_params, learning_rate = 0.05, momentum=0.9)
        updates_exp = [val for key,val in all_updates.items()]
        self.test_params = theano.function(self.inputs_variables+self.inputs_masks+[target1, target2], updates_exp, on_unused_input='warn')
        self.train_func = theano.function(self.inputs_variables + self.inputs_masks + [target1, target2], 
                                        [self.cost, prob1, prob2], updates = all_updates)

        # Compute adam updates for training

        self.prediction = theano.function(self.inputs_variables + self.inputs_masks, [score1, score2])
      

In [ ]:

a = Hierachi_RNN('300','0.0','20','0.25')
a.model_constructor()



In [400]:
v = [[],[],[],[],[],[]]
for i in range(6):
    for j in range(5):
        v[i].append(np.random.randint(100, size=(leng_v[i+j], )))
        
inputs_v = [utils.padding(sentence) for sentence in v]

inputs_m = [utils.mask_generator(sentence) for sentence in v]

targ1 = np.random.randint(2, size=(5,))
targ2 = 1 - targ1
params = a.train_func(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                                  inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5], targ1, targ2)



In [401]:
params[0]

array(7.641996219323214)

In [328]:
score1= a.test2(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                                  inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5])





In [332]:
inputs_v

[array([[28, 54, 37, 81, 45, 85, 43, 30,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 1, 45,  7, 27, 75, 20, 19, 50,  0, 61, 89, 68, 88, 63, 27,  5],
        [49, 47, 95, 73, 22, 82, 58, 60, 94, 66,  6, 38, 87, 38, 67,  0],
        [57, 51,  4, 97, 44, 47, 61, 35, 10, 62, 92,  0,  0,  0,  0,  0],
        [ 1, 61, 44, 99, 33, 92, 71, 81, 57, 10, 59, 26, 39, 91, 66, 29]]),
 array([[93, 81, 48, 80, 88, 74, 81, 96, 98, 67, 59, 72, 51, 93, 17, 78],
        [66, 13, 22, 10, 19, 47, 10, 81, 57, 43, 78, 56, 90, 11, 84,  0],
        [ 4, 33, 74, 10, 97,  2,  0, 86, 93, 51, 52,  0,  0,  0,  0,  0],
        [19, 22, 74, 43, 85, 97, 49, 46, 80, 95, 72, 18, 13, 13, 48, 48],
        [54, 35, 99, 72, 19, 95, 30,  0,  0,  0,  0,  0,  0,  0,  0,  0]]),
 array([[44, 22, 18, 66, 27, 56,  9, 41,  0, 26, 78, 41, 15, 85,  6,  0],
        [ 8, 79, 77, 60,  3, 53, 39, 11, 87, 26, 71,  0,  0,  0,  0,  0],
        [ 8, 87, 11, 57, 76, 87, 51, 82, 90, 10, 56, 16, 14, 14, 13, 23],
        [10, 22, 44, 56, 18, 77, 7

In [178]:
import utils
leng_v = np.random.randint(5, 17, size=(30))

v = [[],[],[],[],[],[]]
for i in range(6):
    for j in range(5):
        v[i].append(np.random.randint(100, size=(leng_v[i+j], )))
        
inputs_v = [utils.padding(sentence) for sentence in v]

inputs_m = [utils.mask_generator(sentence) for sentence in v]

result = a.test2(inputs_v[0], inputs_v[1], inputs_v[2], inputs_v[3], inputs_v[4], inputs_v[5],
                 inputs_m[0], inputs_m[1], inputs_m[2], inputs_m[3], inputs_m[4], inputs_m[5])

In [200]:
print inputs_v[0]
print inputs_m[0]
print result[0][3][10]

[[11  4 55 67 81 81  6 29  0  0  0  0  0  0  0  0]
 [21 36 34 19 78 49  2 19  0 77 75 74 17  9 19  3]
 [ 5 29 11 94 21 70 78 10  2  6 98 42 66 28  8  0]
 [ 3 67 35 72 27 50 47 96 91 89 98  0  0  0  0  0]
 [44 30 99 88 56 79 69 91 34 33 72 41 28 81 74 33]]
[[ 1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.]
 [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.]]
[  6.12901829e-04   1.28206219e-02   1.85810721e-02   1.77323277e-03
  -1.10088005e-02  -5.78050119e-04   9.91322895e-04   1.83338379e-02
  -7.76145479e-03   9.97180350e-03   2.69632479e-03  -1.11936929e-02
  -6.97481570e-03   6.67866176e-03  -6.61085645e-04   8.17177092e-03
  -8.19279369e-03   1.05830154e-02  -1.43372396e-02  -2.16162561e-03
   3.08066795e-03   5.19928999e-03   5.88627218e-04  -7.8830421

In [150]:

IN_ = T.matrix(dtype = 'int64')
MASK = T.matrix(dtype = theano.config.floatX)

IN = IN_.reshape((IN_.shape[0], IN_.shape[1], 1))

wemb = theano.shared(np.random.rand(100, 300))
l_in = lasagne.layers.InputLayer(shape=(None, None, 1))

# Masks input shape ==> (n_batch, n_time_steps)
l_mask = lasagne.layers.InputLayer(shape=(None, None))

#setting gates and cell parameters with specific nonlinearity functions
gate_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                W_hid=lasagne.init.Orthogonal(),
                                                b=lasagne.init.Constant(0.))

cell_parameters = lasagne.layers.recurrent.Gate(W_in=lasagne.init.Orthogonal(), 
                                                W_hid=lasagne.init.Orthogonal(),
                                                # Setting W_cell to None denotes that no cell connection will be used. 
                                                W_cell=None, 
                                                b=lasagne.init.Constant(0.),
                                                # By convention, the cell nonlinearity is tanh in an LSTM. 
                                                nonlinearity=lasagne.nonlinearities.tanh)

# The embedding layers with retieve subtensor from word embedding matrix
l_emb = lasagne.layers.EmbeddingLayer(l_in, input_size=wemb.get_value().shape[0], output_size=wemb.get_value().shape[1], W=wemb)

l_drop = lasagne.layers.DropoutLayer(l_emb, p = 0.0)
# The LSTM layer should have the same mask input in order to avoid padding entries
l_lstm = lasagne.layers.recurrent.LSTMLayer(l_drop, 
                                            num_units=300,
                                            # We need to specify a separate input for masks
                                            mask_input=l_mask,
                                            # Here, we supply the gate parameters for each gate 
                                            ingate=gate_parameters, forgetgate=gate_parameters, 
                                            cell=cell_parameters, outgate=gate_parameters,
                                            # We'll learn the initialization and use gradient clipping 
                                            learn_init=True
                                            )


# The back directional LSTM layers
l_lstm_back = lasagne.layers.recurrent.LSTMLayer(l_drop,
                                                 num_units=300,
                                                 mask_input = l_mask,
                                                 ingate=gate_parameters, forgetgate=gate_parameters, 
                                                 cell=cell_parameters, outgate=gate_parameters,
                                                 # We'll learn the initialization and use gradient clipping 
                                                 learn_init=True,
                                                 backwards=True
                                                )


# Do sum up of bidirectional LSTM results
l_sum = lasagne.layers.ElemwiseSumLayer([l_lstm, l_lstm_back])
out = lasagne.layers.get_output(l_sum, {l_in:IN, l_mask:MASK}, deterministic = True)

test = theano.function([IN_, MASK], out)

In [174]:
result = test(inputs_v[5], inputs_m[5])
print result[4][9]

[  1.41805189e-01   4.65947280e-02  -8.36854174e-02   1.07350674e-01
  -2.37396935e-01  -1.79823246e-01   6.51879467e-02   4.60637026e-03
  -2.09297061e-01  -1.59060352e-01   2.93781539e-01   2.85348498e-01
   8.93948845e-02   5.16647735e-02  -6.63083297e-01  -3.32605698e-01
  -2.13643046e-01  -2.52780222e-02   3.30183841e-01  -1.86089749e-01
   2.64025800e-01   1.07492811e-01   5.57596003e-02  -4.45173354e-01
  -1.89919893e-01   1.83129174e-01   1.55992846e-01   6.95604518e-02
   2.86514647e-01  -4.01425141e-01  -3.06143785e-02  -2.88565510e-01
  -6.13725809e-01  -1.95180382e-01  -2.38842957e-01  -2.73882412e-01
  -1.94990750e-02   1.82266075e-01  -2.67372722e-02   1.89460599e-01
  -3.17284749e-01   1.24459833e-01   1.97643377e-02  -4.07787518e-02
   1.33853864e-01  -1.82464355e-01   8.99734217e-02   3.00601495e-01
   1.07841359e-01   1.00602480e-01  -1.10121118e-01  -1.49100513e-01
   4.55662451e-02   3.19698477e-01  -2.52166452e-01  -5.48263467e-02
  -1.66052223e-01   2.32730964e-01

In [30]:
a = T.matrix()

b = T.sum(a, axis = 1).reshape([-1,1])

c = T.matrix()
d = c / b
test = theano.function([a,c], d)

In [32]:
test(np.array([[1,1]]), np.array([[3,3]]))

array([[ 1.5,  1.5]])

In [224]:
a = T.tensor3()

b = theano.shared
a_ = a.reshape((-1, a.shape[2]))
b_ = b.reshape((1,b.shape[0],b.shape[1]))
b_f = b_ + T.zeros((a_.shape[0], b.shape[0], b.shape[1]))
result = T.batched_dot(a_, b_f).reshape((a.shape[0], a.shape[1], -1))
loss = result.sum()

test = theano.function([a,b], result)
train  = theano.function([a,b], )

In [225]:
a_test = np.arange(3*4*5).reshape((3,4,5))

b_test = np.arange(5*5).reshape((5,5))

testing = test(a_test, b_test)
print testing

ValueError: Shape mismatch: batch sizes unequal. x.shape is (12, 1, 5), y.shape is (1, 5, 5).
Apply node that caused the error: BatchedDot(Reshape{2}.0, Reshape{3}.0)
Toposort index: 10
Inputs types: [TensorType(float64, matrix), TensorType(float64, (True, False, False))]
Inputs shapes: [(12, 5), (1, 5, 5)]
Inputs strides: [(40, 8), (200, 40, 8)]
Inputs values: ['not shown', 'not shown']
Outputs clients: [[Reshape{3}(BatchedDot.0, MakeVector{dtype='int64'}.0)]]

Backtrace when the node is created(use Theano flag traceback.limit=N to make it longer):
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 276, in dispatcher
    return self.dispatch_shell(stream, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 228, in dispatch_shell
    handler(stream, idents, msg)
  File "/usr/local/lib/python2.7/site-packages/ipykernel/kernelbase.py", line 391, in execute_request
    user_expressions, allow_stdin)
  File "/usr/local/lib/python2.7

In [241]:
a = T.tensor3()
b = T.matrix()

loss = T.batched_dot(a, b)
test = theano.function([a,b], loss)
# test = theano.function([inp], loss)



In [242]:
a_test = np.arange(3*4*5).reshape((3,4,5))
b_test = np.arange(3*5).reshape((3,5))
print a_test
print b_test
test(a_test, b_test)






[[[ 0  1  2  3  4]
  [ 5  6  7  8  9]
  [10 11 12 13 14]
  [15 16 17 18 19]]

 [[20 21 22 23 24]
  [25 26 27 28 29]
  [30 31 32 33 34]
  [35 36 37 38 39]]

 [[40 41 42 43 44]
  [45 46 47 48 49]
  [50 51 52 53 54]
  [55 56 57 58 59]]]
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]


array([[   30.,    80.,   130.,   180.],
       [  780.,   955.,  1130.,  1305.],
       [ 2530.,  2830.,  3130.,  3430.]])

In [83]:
a = T.tensor3()
b = T.tensor3()

b_ = b.dimshuffle(0,2,1)
a_ = a[0]
test1 = theano.function([a], a_)
c_ls, _ = theano.scan(lambda i:T.batched_dot(a[i], b_[i]), sequences=[T.arange(a.shape[0])])
test = theano.function([a,b], c_ls)


In [245]:
a = np.array([1,2,3,4])
b = np.array([1,2,3,3])
c = np.all(a-b==0)
print c

False


In [95]:
a = np.arange(3 * 4 * 5).reshape(3, 4, 5).astype('float32')
b = np.arange(3 * 5).reshape(3, 5).astype('float32')

A = T.ftensor3()
B = T.fmatrix()

# out1 = T.tensordot(A,B)
out = (A * B.dimshuffle(0, 'x', 1)).sum(2)
print a
print b
out.eval({A: a, B: b})
# out1.eval({A:a, B:b})


[[[  0.   1.   2.   3.   4.]
  [  5.   6.   7.   8.   9.]
  [ 10.  11.  12.  13.  14.]
  [ 15.  16.  17.  18.  19.]]

 [[ 20.  21.  22.  23.  24.]
  [ 25.  26.  27.  28.  29.]
  [ 30.  31.  32.  33.  34.]
  [ 35.  36.  37.  38.  39.]]

 [[ 40.  41.  42.  43.  44.]
  [ 45.  46.  47.  48.  49.]
  [ 50.  51.  52.  53.  54.]
  [ 55.  56.  57.  58.  59.]]]
[[  0.   1.   2.   3.   4.]
 [  5.   6.   7.   8.   9.]
 [ 10.  11.  12.  13.  14.]]


array([[   30.,    80.,   130.,   180.],
       [  780.,   955.,  1130.,  1305.],
       [ 2530.,  2830.,  3130.,  3430.]], dtype=float32)

In [263]:
a = T.matrix()
a_ = T.nnet.softmax(a)
b = T.imatrix()

c = lasagne.objectives.categorical_crossentropy(a_, b)
test = theano.function([a, b], c)

In [274]:
import cPickle as pickle
dictionary = pickle.load(open('../../data/pickles/index_wemb_matrix.pkl'))

In [275]:
dictionary[0]

array([ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0

In [390]:
x = T.matrix()

e_x = x.max(axis=1, keepdims=True)

test = theano.function([x], e_x)

In [391]:
a = T.matrix()
b = T.





array([[  3.],
       [  7.],
       [ 11.]])